# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key

from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data2/cities2.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,-36.12,93,100,0.89,RU,1708267974
1,1,mil'kovo,55.6078,37.7956,-1.73,80,100,4.92,RU,1708267975
2,2,gargalianoi,37.0667,21.6333,15.75,69,0,4.36,GR,1708267976
3,3,akureyri,65.6835,-18.0878,5.96,70,20,5.66,IS,1708267977
4,4,esperanza,19.5847,-70.9849,26.32,44,4,1.70,DO,1708267978


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=27)\
                            &(city_data_df['Max Temp']>=21)\
                            &(city_data_df['Wind Speed']<=4.5)\
                            &(city_data_df['Cloudiness']==0)\
                            ,:]
print(len(ideal_df))


# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

8


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,ciudad lazaro cardenas,17.9583,-102.2000,23.40,68,0,0.99,MX,1708267998
30,30,san luis de la loma,17.2706,-100.8939,24.66,59,0,1.06,MX,1708268006
104,104,puerto real,36.5282,-6.1901,21.09,56,0,3.09,ES,1708268083
148,148,chiang klang,19.2938,100.8617,23.37,33,0,0.65,TH,1708268129
191,191,lazaro cardenas,17.9583,-102.2000,23.40,68,0,0.99,MX,1708267998


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,susuman,RU,62.7833,148.1667,93,
1,mil'kovo,RU,55.6078,37.7956,80,
2,gargalianoi,GR,37.0667,21.6333,69,
3,akureyri,IS,65.6835,-18.0878,70,
4,esperanza,DO,19.5847,-70.9849,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
   'categories': 'accommodation.hotel',
   'filter': '',
   'bias': '',
   'api_key': geoapify_key,
   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    if index >= 20:
        break
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
susuman - nearest hotel: No hotel found
mil'kovo - nearest hotel: Дрозды
gargalianoi - nearest hotel: Messinian Horizons
akureyri - nearest hotel: Hótel Norðurland
esperanza - nearest hotel: Hotel Novus Caoba
hauterive - nearest hotel: Auberge Bourbonnaise
mayenne - nearest hotel: Hôtel L'Entracte
port-aux-francais - nearest hotel: Keravel
brisas de zicatela - nearest hotel: Casa de Olas
klyuchi - nearest hotel: No hotel found
chilca - nearest hotel: Hotel Benvenuto
tamanrasset - nearest hotel: فندق الأمان
talnakh - nearest hotel: Талнах
arraial do cabo - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
hamilton - nearest hotel: North Vista Manor
sao felix do xingu - nearest hotel: Hotel Terraço
margaret river - nearest hotel: Margaret River Hotel
invercargill - nearest hotel: The Grand


,City,Country,Lat,Lng,Humidity,Hotel Name
0,susuman,RU,62.7833,148.1667,93,No hotel found
1,mil'kovo,RU,55.6078,37.7956,80,Дрозды
2,gargalianoi,GR,37.0667,21.6333,69,Messinian Horizons
3,akureyri,IS,65.6835,-18.0878,70,Hótel Norðurland
4,esperanza,DO,19.5847,-70.9849,44,Hotel Novus Caoba
...,...,...,...,...,...,...
569,nar'yan-mar,RU,67.6713,53.0870,93,
570,charters towers,AU,-20.1000,146.2667,84,
571,perry,US,37.2506,-83.1999,37,
572,peringat,MY,6.0333,102.2833,91,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.head(20).hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            scale= 2,
            frame_width = 600, 
            frame_height = 600, 
            color='City',
            alpha=0.4,
            hover=True,
            hover_cols=['City', 'Hotel Name', 'Country'],
            use_index=False,
            s='Humidity'
            )

# Display the map
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)